In [12]:
import sys
import os
import random
from operator import add, mul
from pyspark import SparkContext, SparkConf
from pyspark import SparkFiles
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import Row
from pyspark.sql.functions import col,sum


In [2]:
spark = SparkSession.builder.appName("M6-CA1-Instacart-TGA").getOrCreate() # singleton instance

### 1.Load data into Spark DataFrame


In [3]:
# hdfs dfs -put orders.csv /user/edureka_672184/use_cases/instacart_csv/
print("\n1. Load Data into Spark DataFrame\n")
aisles_df = spark.read.csv("/user/edureka_672184/use_cases/instacart_csv/aisles.csv", header=True)
department_df = spark.read.csv("/user/edureka_672184/use_cases/instacart_csv/departments.csv", header=True)
products_df = spark.read.csv("/user/edureka_672184/use_cases/instacart_csv/products.csv", header=True)
orders_df = spark.read.csv("/user/edureka_672184/use_cases/instacart_csv/orders.csv", header=True)
orders_products_prior_df = spark.read.csv("/user/edureka_672184/use_cases/instacart_csv/order_products__prior.csv", header=True)
orders_products_train_df = spark.read.csv("/user/edureka_672184/use_cases/instacart_csv/order_products__train.csv", header=True)
#sample_submission_df = spark.read.csv("/user/edureka_672184/use_cases/instacart_csv/sample_submission.csv", header=True)


1. Load Data into Spark DataFrame



In [4]:
print("Showing products")
print(products_df.show(5))
print("Showing orders")
print(orders_df.show(5))
print("orders products train")
print(orders_products_train_df.show(5))

Showing products
+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
+----------+--------------------+--------+-------------+
only showing top 5 rows

None
Showing orders
+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|               08|                  null|
| 2398795|      1|   prior|           2|        3|       

### 2.Merge all the data frames based on the common key and create a single DataFrame 


In [5]:
print("\n2. Merge all the data frames\n")
prod_dept_aisle = aisles_df \
.join(products_df, on="aisle_id", how='inner') \
.join(department_df, on="department_id", how='inner')


2. Merge all the data frames



In [6]:
print("products, Department and Aisle combined")
print(prod_dept_aisle.show(5))

products, Department and Aisle combined
+-------------+--------+--------------------+----------+--------------------+----------+
|department_id|aisle_id|               aisle|product_id|        product_name|department|
+-------------+--------+--------------------+----------+--------------------+----------+
|           20|       1|prepared soups sa...|     49445|   Baked Potato Soup|      deli|
|           20|       1|prepared soups sa...|     49056|Organic Blend Me ...|      deli|
|           20|       1|prepared soups sa...|     48556|    Red Potato Salad|      deli|
|           20|       1|prepared soups sa...|     48474| Deli Fresh Coleslaw|      deli|
|           20|       1|prepared soups sa...|     47979|Butternut Squash ...|      deli|
+-------------+--------+--------------------+----------+--------------------+----------+
only showing top 5 rows

None


In [7]:
print("products, Department and Aisle and other tables combined")

prod_order_prior = orders_df.join(orders_products_prior_df, on="order_id", how='inner')
prod_order_train = orders_df.join(orders_products_train_df, on="order_id", how='inner')
prod_order = prod_order_prior.union(prod_order_train)

products, Department and Aisle and other tables combined


In [8]:
instacart_full_df = prod_dept_aisle.join(prod_order, on="product_id", how="inner")

In [9]:
print("Showing 2 rows from the complete dataframe")
instacart_full_df.show(2)

Showing 2 rows from the complete dataframe
+----------+-------------+--------+--------------------+--------------------+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+
|product_id|department_id|aisle_id|               aisle|        product_name|department|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|add_to_cart_order|reordered|
+----------+-------------+--------+--------------------+--------------------+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+
|     10096|            1|      58|frozen breads doughs|Corn Meal Pizza C...|    frozen| 1047552| 156272|   prior|          29|        3|               11|                   6.0|                3|        1|
|     10096|            1|      58|frozen breads doughs|Corn Meal Pizza C...|    frozen| 1832904| 156059|   prior|          35|  

In [10]:
print("showing columns of the complete dataframe")
print(instacart_full_df.columns)

showing columns of the complete dataframe
['product_id', 'department_id', 'aisle_id', 'aisle', 'product_name', 'department', 'order_id', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'add_to_cart_order', 'reordered']


### 3.Check missing data


In [13]:
# showing nulls
print("\n3. Check missing data\n")

print("Showing nulls per column ")
print(instacart_full_df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in instacart_full_df.columns)).show())


3. Check missing data

Showing nulls per column 
+----------+-------------+--------+-----+------------+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+
|product_id|department_id|aisle_id|aisle|product_name|department|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|add_to_cart_order|reordered|
+----------+-------------+--------+-----+------------+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+
|         0|            0|       0|    0|           0|         0|       0|      0|       0|           0|        0|                0|               2078068|                0|        0|
+----------+-------------+--------+-----+------------+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+

None


Days since prior order has 2 million nulls. Other columns are good.

In [14]:
print("Days since prior order has 2 million nulls. Other columns are good.")
print("Size of full data is: ", instacart_full_df.count())

Days since prior order has 2 million nulls. Other columns are good.
('Size of full data is: ', 33819103)


### 4.List the most ordered products (top 10)


In [15]:
print("\n4. List the most ordered products (top 10) \n")
print(instacart_full_df.groupBy(["product_id", "product_name"]).count().orderBy('count', ascending=False).show(10, False))


4. List the most ordered products (top 10) 

+----------+----------------------+------+
|product_id|product_name          |count |
+----------+----------------------+------+
|24852     |Banana                |491291|
|13176     |Bag of Organic Bananas|394930|
|21137     |Organic Strawberries  |275577|
|21903     |Organic Baby Spinach  |251705|
|47209     |Organic Hass Avocado  |220877|
|47766     |Organic Avocado       |184224|
|47626     |Large Lemon           |160792|
|16797     |Strawberries          |149445|
|26209     |Limes                 |146660|
|27845     |Organic Whole Milk    |142813|
+----------+----------------------+------+
only showing top 10 rows

None


### 5.Do people usually reorder the same previous ordered products?


In [16]:
print("\n5. Do people usually reorder the same previous ordered products? \n")
print(instacart_full_df.groupBy("reordered").count().orderBy('count', ascending=False).show())


5. Do people usually reorder the same previous ordered products? 

+---------+--------+
|reordered|   count|
+---------+--------+
|        1|19955360|
|        0|13863743|
+---------+--------+

None


### 6.List most reordered products


In [17]:
print("\n6. List most reordered products \n")
print(instacart_full_df.filter(col("reordered") == 1). \
      groupBy(["product_id", "product_name"]). \
      count().orderBy('count', ascending=False). \
      show(10, False))


6. List most reordered products 

+----------+----------------------+------+
|product_id|product_name          |count |
+----------+----------------------+------+
|24852     |Banana                |415166|
|13176     |Bag of Organic Bananas|329275|
|21137     |Organic Strawberries  |214448|
|21903     |Organic Baby Spinach  |194939|
|47209     |Organic Hass Avocado  |176173|
|47766     |Organic Avocado       |140270|
|27845     |Organic Whole Milk    |118684|
|47626     |Large Lemon           |112178|
|27966     |Organic Raspberries   |109688|
|16797     |Strawberries          |104588|
+----------+----------------------+------+
only showing top 10 rows

None


### 7.Most important department and aisle (by number of products)


In [18]:
print("\n7. Most important department and aisle (by number of products) \n")
print(prod_dept_aisle. \
      groupby(["aisle_id", "aisle", "department_id", "department"]). \
      count().orderBy('count', ascending=False).show(1))


7. Most important department and aisle (by number of products) 

+--------+-------+-------------+----------+-----+
|aisle_id|  aisle|department_id|department|count|
+--------+-------+-------------+----------+-----+
|     100|missing|           21|   missing| 1258|
+--------+-------+-------------+----------+-----+
only showing top 1 row

None


### 8.Get the Top 10 departments


In [19]:
print("\n8. Get the Top 10 departments) \n")
print(prod_dept_aisle. \
      groupby(["department_id", "department"]). \
      count().orderBy('count', ascending=False).show(10, False))


8. Get the Top 10 departments) 

+-------------+---------------+-----+
|department_id|department     |count|
+-------------+---------------+-----+
|11           |personal care  |6563 |
|19           |snacks         |6264 |
|13           |pantry         |5371 |
|7            |beverages      |4365 |
|1            |frozen         |4007 |
|16           |dairy eggs     |3449 |
|17           |household      |3084 |
|15           |canned goods   |2092 |
|9            |dry goods pasta|1858 |
|4            |produce        |1684 |
+-------------+---------------+-----+
only showing top 10 rows

None


### 9.List top 10 products ordered in the morning (6 AM to 11 AM)


In [20]:
print("\n9. List top 10 products ordered in the morning (6 AM to 11 AM)) \n")
# 6 AM in 24 hour format = 6
# 11 AM in 24 hour format = 11

print(instacart_full_df.filter((col("order_hour_of_day") >= 6) & (col("order_hour_of_day") <= 11)). \
      groupBy(["product_id", "product_name"]). \
      count().orderBy('count', ascending=False).show(10, False))


9. List top 10 products ordered in the morning (6 AM to 11 AM)) 

+----------+----------------------+------+
|product_id|product_name          |count |
+----------+----------------------+------+
|24852     |Banana                |169965|
|13176     |Bag of Organic Bananas|135417|
|21137     |Organic Strawberries  |92499 |
|21903     |Organic Baby Spinach  |82578 |
|47209     |Organic Hass Avocado  |72545 |
|47766     |Organic Avocado       |59603 |
|47626     |Large Lemon           |53479 |
|16797     |Strawberries          |52155 |
|27966     |Organic Raspberries   |49751 |
|27845     |Organic Whole Milk    |49747 |
+----------+----------------------+------+
only showing top 10 rows

None


### 10.Create a spark-submit application for the same and print the findings inthelog

In [ ]:
!spark2-submit /mnt/home/edureka_672184/m6ca1q10.py

19/07/25 06:01:31 INFO spark.SparkContext: Running Spark version 2.1.0.cloudera2
19/07/25 06:01:32 INFO spark.SecurityManager: Changing view acls to: edureka_672184
19/07/25 06:01:32 INFO spark.SecurityManager: Changing modify acls to: edureka_672184
19/07/25 06:01:32 INFO spark.SecurityManager: Changing view acls groups to: 
19/07/25 06:01:32 INFO spark.SecurityManager: Changing modify acls groups to: 
19/07/25 06:01:32 INFO spark.SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(edureka_672184); groups with view permissions: Set(); users  with modify permissions: Set(edureka_672184); groups with modify permissions: Set()
19/07/25 06:01:33 INFO util.Utils: Successfully started service 'sparkDriver' on port 39366.
19/07/25 06:01:33 INFO spark.SparkEnv: Registering MapOutputTracker
19/07/25 06:01:33 INFO spark.SparkEnv: Registering BlockManagerMaster
19/07/25 06:01:33 INFO storage.BlockManagerMasterEndpoint: Using org.apache.s

19/07/25 06:01:57 INFO internal.SharedState: spark.sql.warehouse.dir is not set, but hive.metastore.warehouse.dir is set. Setting spark.sql.warehouse.dir to the value of hive.metastore.warehouse.dir ('/user/hive/warehouse').
19/07/25 06:01:57 INFO internal.SharedState: Warehouse path is '/user/hive/warehouse'.


19/07/25 06:01:57 INFO hive.HiveUtils: Initializing HiveMetastoreConnection version 1.1.0 using file:/opt/cloudera/parcels/CDH-5.11.1-1.cdh5.11.1.p0.4/lib/hadoop/../hive/lib/commons-logging-1.1.3.jar:file:/opt/cloudera/parcels/CDH-5.11.1-1.cdh5.11.1.p0.4/lib/hadoop/../hive/lib/hive-exec-1.1.0-cdh5.11.1.jar:file:/opt/cloudera/parcels/CDH-5.11.1-1.cdh5.11.1.p0.4/lib/hadoop/../hive/lib/hive-exec.jar:file:/opt/cloudera/parcels/CDH-5.11.1-1.cdh5.11.1.p0.4/lib/hadoop/../hive/lib/hive-jdbc-1.1.0-cdh5.11.1-standalone.jar:file:/opt/cloudera/parcels/CDH-5.11.1-1.cdh5.11.1.p0.4/lib/hadoop/../hive/lib/hive-jdbc-1.1.0-cdh5.11.1.jar:file:/opt/cloudera/parcels/CDH-5.11.1-1.cdh5.11.1.p0.4/lib/hadoop/../hive/lib/hive-jdbc-standalone.jar:file:/opt/cloudera/parcels/CDH-5.11.1-1.cdh5.11.1.p0.4/lib/hadoop/../hive/lib/hive-jdbc.jar:file:/opt/cloudera/parcels/CDH-5.11.1-1.cdh5.11.1.p0.4/lib/hadoop/../hive/lib/hive-metastore-1.1.0-cdh5.11.1.jar:file:/opt/cloudera/parcels/CDH-5.11.1-1.cdh5.11.1.p0.4/lib/hadoop

19/07/25 06:01:59 INFO session.SessionState: Created local directory: /tmp/766bd394-5fc4-40cc-b322-ddbf1ae46749_resources
19/07/25 06:01:59 INFO session.SessionState: Created HDFS directory: /tmp/hive/edureka_672184/766bd394-5fc4-40cc-b322-ddbf1ae46749
19/07/25 06:01:59 INFO session.SessionState: Created local directory: /tmp/edureka_672184/766bd394-5fc4-40cc-b322-ddbf1ae46749
19/07/25 06:01:59 INFO session.SessionState: Created HDFS directory: /tmp/hive/edureka_672184/766bd394-5fc4-40cc-b322-ddbf1ae46749/_tmp_space.db
19/07/25 06:01:59 INFO session.SessionState: No Tez session required at this point. hive.execution.engine=mr.
19/07/25 06:01:59 INFO client.HiveClientImpl: Warehouse location for Hive client (version 1.1.0) is /user/hive/warehouse
19/07/25 06:02:00 INFO hive.metastore: Trying to connect to metastore with URI thrift://ip-20-0-21-161.ec2.internal:9083
19/07/25 06:02:00 INFO hive.metastore: Opened a connection to metastore, current connections: 1
19/07/25 06:02:00 INFO hive

19/07/25 06:02:13 INFO storage.BlockManagerInfo: Removed broadcast_1_piece0 on 20.0.31.136:45772 in memory (size: 2.1 KB, free: 93.2 MB)
19/07/25 06:02:13 INFO storage.BlockManagerInfo: Removed broadcast_1_piece0 on ip-20-0-31-39.ec2.internal:44829 in memory (size: 2.1 KB, free: 366.3 MB)
19/07/25 06:02:13 INFO storage.BlockManagerInfo: Removed broadcast_0_piece0 on 20.0.31.136:45772 in memory (size: 29.1 KB, free: 93.3 MB)
19/07/25 06:02:13 INFO storage.BlockManagerInfo: Removed broadcast_0_piece0 on ip-20-0-31-39.ec2.internal:44829 in memory (size: 29.1 KB, free: 366.3 MB)
19/07/25 06:02:13 INFO storage.BlockManagerInfo: Removed broadcast_2_piece0 on 20.0.31.136:45772 in memory (size: 29.1 KB, free: 93.3 MB)
19/07/25 06:02:13 INFO storage.BlockManagerInfo: Removed broadcast_2_piece0 on ip-20-0-31-39.ec2.internal:42194 in memory (size: 29.1 KB, free: 366.3 MB)
19/07/25 06:02:13 INFO storage.BlockManagerInfo: Removed broadcast_3_piece0 on 20.0.31.136:45772 in memory (size: 2.1 KB, free

...

19/07/25 06:12:54 INFO scheduler.TaskSetManager: Starting task 146.0 in stage 76.0 (TID 4051, ip-20-0-31-39.ec2.internal, executor 1, partition 146, PROCESS_LOCAL, 5904 bytes)
19/07/25 06:12:54 INFO scheduler.TaskSetManager: Starting task 147.0 in stage 76.0 (TID 4052, ip-20-0-31-39.ec2.internal, executor 2, partition 147, PROCESS_LOCAL, 5904 bytes)
19/07/25 06:12:54 INFO scheduler.TaskSetManager: Finished task 144.0 in stage 76.0 (TID 4049) in 3 ms on ip-20-0-31-39.ec2.internal (executor 1) (151/200)
19/07/25 06:12:54 INFO scheduler.TaskSetManager: Finished task 145.0 in stage 76.0 (TID 4050) in 3 ms on ip-20-0-31-39.ec2.internal (executor 2) (152/200)
19/07/25 06:12:54 INFO scheduler.TaskSetManager: Starting task 148.0 in stage 76.0 (TID 4053, ip-20-0-31-39.ec2.internal, executor 1, partition 148, PROCESS_LOCAL, 5904 bytes)
19/07/25 06:12:54 INFO scheduler.TaskSetManager: Finished task 146.0 in stage 76.0 (TID 4051) in 3 ms on ip-20-0-31-39.ec2.internal (executor 1) (153/200)
19/07/2

19/07/25 06:12:54 INFO scheduler.TaskSetManager: Starting task 185.0 in stage 76.0 (TID 4085, ip-20-0-31-39.ec2.internal, executor 1, partition 185, PROCESS_LOCAL, 5904 bytes)
19/07/25 06:12:54 INFO scheduler.TaskSetManager: Finished task 183.0 in stage 76.0 (TID 4083) in 3 ms on ip-20-0-31-39.ec2.internal (executor 1) (185/200)
19/07/25 06:12:54 INFO scheduler.TaskSetManager: Starting task 186.0 in stage 76.0 (TID 4086, ip-20-0-31-39.ec2.internal, executor 2, partition 186, PROCESS_LOCAL, 5904 bytes)
19/07/25 06:12:54 INFO scheduler.TaskSetManager: Finished task 184.0 in stage 76.0 (TID 4084) in 4 ms on ip-20-0-31-39.ec2.internal (executor 2) (186/200)
19/07/25 06:12:54 INFO scheduler.TaskSetManager: Starting task 187.0 in stage 76.0 (TID 4087, ip-20-0-31-39.ec2.internal, executor 1, partition 187, PROCESS_LOCAL, 5904 bytes)
19/07/25 06:12:54 INFO scheduler.TaskSetManager: Finished task 185.0 in stage 76.0 (TID 4085) in 3 ms on ip-20-0-31-39.ec2.internal (executor 1) (187/200)
19/07/2

19/07/25 06:12:54 INFO spark.ContextCleaner: Cleaned accumulator 100847
19/07/25 06:12:54 INFO spark.ContextCleaner: Cleaned accumulator 100836
19/07/25 06:12:54 INFO spark.ContextCleaner: Cleaned accumulator 100837
19/07/25 06:12:54 INFO spark.ContextCleaner: Cleaned accumulator 100840
19/07/25 06:12:54 INFO spark.ContextCleaner: Cleaned accumulator 100839
19/07/25 06:12:54 INFO spark.ContextCleaner: Cleaned accumulator 100841
19/07/25 06:12:54 INFO spark.ContextCleaner: Cleaned accumulator 100838
19/07/25 06:12:54 INFO spark.ContextCleaner: Cleaned accumulator 100845
19/07/25 06:12:54 INFO memory.MemoryStore: Block broadcast_155 stored as values in memory (estimated size 330.3 KB, free 83.8 MB)
19/07/25 06:12:54 INFO memory.MemoryStore: Block broadcast_156 stored as values in memory (estimated size 330.3 KB, free 83.5 MB)
19/07/25 06:12:54 INFO memory.MemoryStore: Block broadcast_155_piece0 stored as bytes in memory (estimated size 31.0 KB, free 83.5 MB)
19/07/25 06:12:54 INFO storag

19/07/25 06:12:54 INFO codegen.CodeGenerator: Code generated in 8.948393 ms
19/07/25 06:12:54 INFO memory.MemoryStore: Block broadcast_162 stored as values in memory (estimated size 330.3 KB, free 69.7 MB)
19/07/25 06:12:54 INFO memory.MemoryStore: Block broadcast_162_piece0 stored as bytes in memory (estimated size 31.0 KB, free 69.7 MB)
19/07/25 06:12:54 INFO storage.BlockManagerInfo: Added broadcast_162_piece0 in memory on 20.0.31.136:45772 (size: 31.0 KB, free: 89.0 MB)
19/07/25 06:12:54 INFO spark.SparkContext: Created broadcast 162 from showString at NativeMethodAccessorImpl.java:0
19/07/25 06:12:54 INFO execution.FileSourceScanExec: Planning scan with bin packing, max size: 56581474 bytes, open cost is considered as scanning 4194304 bytes.
19/07/25 06:12:54 INFO storage.BlockManagerInfo: Removed broadcast_158_piece0 on 20.0.31.136:45772 in memory (size: 5.3 KB, free: 89.0 MB)
19/07/25 06:12:54 INFO storage.BlockManagerInfo: Removed broadcast_158_piece0 on ip-20-0-31-39.ec2.inter

19/07/25 06:12:54 INFO storage.BlockManagerInfo: Added broadcast_160_piece0 in memory on ip-20-0-31-39.ec2.internal:42194 (size: 3.0 MB, free: 362.1 MB)
19/07/25 06:12:54 INFO storage.BlockManagerInfo: Added broadcast_159_piece0 in memory on ip-20-0-31-39.ec2.internal:42194 (size: 473.0 B, free: 362.1 MB)
19/07/25 06:12:54 INFO storage.BlockManagerInfo: Added broadcast_161_piece0 in memory on ip-20-0-31-39.ec2.internal:42194 (size: 31.0 KB, free: 362.1 MB)
19/07/25 06:12:55 INFO scheduler.TaskSetManager: Starting task 1.0 in stage 80.0 (TID 4103, ip-20-0-31-39.ec2.internal, executor 2, partition 1, RACK_LOCAL, 6627 bytes)
19/07/25 06:12:55 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 79.0 (TID 4101) in 192 ms on ip-20-0-31-39.ec2.internal (executor 2) (1/1)
19/07/25 06:12:55 INFO cluster.YarnScheduler: Removed TaskSet 79.0, whose tasks have all completed, from pool 
19/07/25 06:12:55 INFO scheduler.DAGScheduler: ShuffleMapStage 79 (showString at NativeMethodAccessorImpl.ja